In [1]:
#Dependencies
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests
from splinter import Browser

In [2]:
#Initialize browser
#Change chromedriver path when uploading to server

executable_path = {'executable_path': r"C:\Users\slapp\Desktop\Misc_class\chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)


In [4]:
#define initial url and direct browser
url = 'https://www.indeed.com/jobs?q=Data+Scientist&explvl=entry_level&fromage=3'
browser.visit(url)

In [5]:
#Make some soup
html = browser.html
soup = BeautifulSoup(html, 'lxml')
browser.quit()

In [6]:
#Scrap all postings on page into a list. Run through loop to click on next page
postings = soup.find_all( class_= 'jobsearch-SerpJobCard')

In [7]:
postings[0]

<div class="jobsearch-SerpJobCard unifiedRow row result clickcard" data-ci="133697537" data-empn="377656857425241" data-jk="00ba1a22ba67ffd2" id="pj_00ba1a22ba67ffd2">
<style>
.jobcard_logo{margin:6px 0}.jobcard_logo img{width:auto;max-width:80px;max-height:30px}
</style>
<h2 class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0D3UvD5kBSgX9r9tFJCI4OL-41vvae__bcle4uMSq30h36AcPbHjl9bcVWQE4zSRT4EOgpM6LkPra63lxoEtVae-7wD-oUwUbBd1OZLV0EDBfVsDapnyVIX5sAVVSZi1JFDgw1-QU0Z_6h_hb3hkF0kGSe2adBRcLe-9dykDVUwA9_LgUuV-p7vSPZG1PunCyO5w1fAvs1ggIIq2mkPEYgnJVlrQ5CLibY1nFSKM6cLiGCUEGNpg2leVcuCMiaeOYUhdvWHD5XjfIsHzF_-MsvEhTLcDZMXY-fSUjHJnVgwdBz4VJgRccTLSYTpL8y9bqu_ylkdvGE7w8j9vlvg8WPFLXshv8rJhlMghplOqgUPVhUCAfGQj7hYnqNQo4l3nAzbxahAChY0jXXSMGDzm7_IAGyMh5-E9gBx9Am2kxLjk5QvFByJL7GW53aZJ3TS_gdDwwwyCwjQl6TrCpzOg3jPdHxYO4RHb1QDZ3-JeMd4GG0aFT_qnVxaR-Hok_cr4eN-bTgyVtMm5iUYAcGMa0dNmZqs3Ew7w1DFSVTl6rPCqdw3HdARnCg5B6lP4i9CSFw=&amp;p=0&amp;fvj=0&amp;vjs=3"

In [8]:
len(postings)

14

In [9]:
#Scrape url and add domaine
f"https://www.indeed.com{postings[0].find('h2').find(class_ = 'jobtitle')['href']}"


'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D3UvD5kBSgX9r9tFJCI4OL-41vvae__bcle4uMSq30h36AcPbHjl9bcVWQE4zSRT4EOgpM6LkPra63lxoEtVae-7wD-oUwUbBd1OZLV0EDBfVsDapnyVIX5sAVVSZi1JFDgw1-QU0Z_6h_hb3hkF0kGSe2adBRcLe-9dykDVUwA9_LgUuV-p7vSPZG1PunCyO5w1fAvs1ggIIq2mkPEYgnJVlrQ5CLibY1nFSKM6cLiGCUEGNpg2leVcuCMiaeOYUhdvWHD5XjfIsHzF_-MsvEhTLcDZMXY-fSUjHJnVgwdBz4VJgRccTLSYTpL8y9bqu_ylkdvGE7w8j9vlvg8WPFLXshv8rJhlMghplOqgUPVhUCAfGQj7hYnqNQo4l3nAzbxahAChY0jXXSMGDzm7_IAGyMh5-E9gBx9Am2kxLjk5QvFByJL7GW53aZJ3TS_gdDwwwyCwjQl6TrCpzOg3jPdHxYO4RHb1QDZ3-JeMd4GG0aFT_qnVxaR-Hok_cr4eN-bTgyVtMm5iUYAcGMa0dNmZqs3Ew7w1DFSVTl6rPCqdw3HdARnCg5B6lP4i9CSFw=&p=0&fvj=0&vjs=3'

In [10]:
#Scrape job title
postings[0].find('h2').find(class_ = 'jobtitle').text.replace('\n', '')


'Data Scientist -'

In [11]:
#scrape company name
postings[0].find(class_ = 'company').text.replace('\n', '')

'Blue Owl'

In [12]:
#scrape location (split into city and state later)
location = postings[0].find(class_ = 'recJobLoc')['data-rc-loc']

In [13]:
#splitting location into city and state. try except to include remote work
cities = []
states = []
for x in range(0, len(location)):
    citystate = location[x].split(', ')
    cities.append(citystate[0])
    try:
        states.append(citystate[1])
    except IndexError:
        states.append(citystate[0])
    #print(cities[x] + states[x])

In [14]:
#scrape date added (current day - "days posted ago")
postings[0].find(class_='date').text

'1 day ago'

In [15]:
#Convert result to int
try:
    testdate = int(postings[0].find(class_='date').text.split(' ')[0])
except ValueError:
    testdate = 0

In [16]:
#subtract from current date w/ date time
d = datetime.today() - timedelta(days=testdate)
d

datetime.datetime(2020, 6, 23, 12, 13, 22, 930448)

In [17]:
#scrape salary, add null if not included
salaryrange = []
try:
    salaryrange.append(postings[1].find(class_ = 'salaryText').text.replace('\n', ''))
except AttributeError:
    salaryrange.append('NaN')
salaryrange

['NaN']

In [26]:
#scrape job abstract
postings[11].find(class_ = 'summary').text.strip().replace('\n', ' ')

'Collaborate and consult with data scientists and subject-area experts on statistics and data analytics problems. Highly skilled in R or Python.'

In [27]:
#Collect all above data for each entry with a loop
post_url = []
job_title = []
company = [] 
city = []
state = []
date_added = []
salaryrange = []
abstract = []

        
for x in range(0, len(postings)):
    post_url.append(f"https://www.indeed.com{postings[x].find('h2').find(class_ = 'jobtitle')['href']}")
    job_title.append(postings[x].find('h2').find(class_ = 'jobtitle').text.replace('\n', ''))
    company.append(postings[x].find(class_ = 'company').text.replace('\n', ''))
    location = postings[x].find(class_ = 'recJobLoc')['data-rc-loc'].split(', ')
    city.append(location[0])
    try:
        state.append(location[1])
    except IndexError:
        state.append(location[0])
    try:
        temp_date = int(postings[x].find(class_='date').text.split(' ')[0])
    except ValueError:
        temp_date = 0
    date_added.append(datetime.today() - timedelta(days=testdate))
    try:
        salaryrange.append(postings[x].find(class_ = 'salaryText').text.replace('\n', ''))
    except AttributeError:
        salaryrange.append('NaN')
    abstract.append(postings[x].find(class_ = 'summary').text.strip().replace('\n', ' '))

In [28]:
abstract

['We’re using rich customer insights, advanced technology and data science to build our cloud-native InsurTech solution.',
 'Experience with some machine learning methodologies such as Neural Networks, SVM, Random Forest, Natural Language Processing. Job Types: Full-time, Contract.',
 'Study Preparation: Meets with CRA, R&D, Instrumentation scientists, department data collection programmers, and data entry to find the best way of capturing the…',
 'The RiceTec Data Scientist will need to Design strategies and implement algorithms to analyze and leverage data; assessing the effectiveness and accuracy of…',
 'Demonstrated experience and familiarity in working with 3-D geospatial data, data registration and big data analysis. Publish and present research results.',
 'Address machine learning problems around recommendation engines, possibly using methods including tensor factorization, auto encoders, temporal models, deep…',
 'Provide data modeling, mining, pattern analysis, data visualiza

In [44]:
#loop through pages
base_url = 'https://www.indeed.com/jobs?q=Data+Scientist&explvl=entry_level&fromage=3&start='
post_url = []
job_title = []
company = [] 
city = []
state = []
date_added = []
salaryrange = []
abstract = []


for x in range(0, 100, 10):
    test_url = f"{base_url}{x}"
    page = requests.get(test_url).content
    soup = BeautifulSoup(page)
    for x in range(0, len(postings)):
        post_url.append(f"https://www.indeed.com{postings[x].find('h2').find(class_ = 'jobtitle')['href']}")
        job_title.append(postings[x].find('h2').find(class_ = 'jobtitle').text.replace('\n', ''))
        company.append(postings[x].find(class_ = 'company').text.replace('\n', ''))
        location = postings[x].find(class_ = 'recJobLoc')['data-rc-loc'].split(', ')
        city.append(location[0])
        try:
            state.append(location[1])
        except IndexError:
            state.append(location[0])
        try:
            temp_date = int(postings[x].find(class_='date').text.split(' ')[0])
        except ValueError:
            temp_date = 0
        date_added.append(datetime.today() - timedelta(days=testdate))
        try:
            salaryrange.append(postings[x].find(class_ = 'salaryText').text.replace('\n', ''))
        except AttributeError:
            salaryrange.append('NaN')
        abstract.append(postings[x].find(class_ = 'summary').text.strip().replace('\n', ' '))



In [45]:
#Create df with all data and write to csv
df = pd.DataFrame(list(zip(post_url, job_title, company, city, state, date_added, salaryrange, abstract)),
                 columns = ['post_url', 'job_title', 'company', 'city', 'state', 'date_added', 'salary_range', 'abstract'])
df.tail(15)

,post_url,job_title,company,city,state,date_added,salary_range,abstract
125,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"AVP, AML Statistician - Scenario Development R...",Citi,Tampa,FL,2020-06-23 13:00:14.059926,NaN,Integrates subject matter and industry experti...
126,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Data Scientist -,Blue Owl,San Francisco,CA,2020-06-23 13:00:14.794885,"$250,000 - $375,000 a year","We’re using rich customer insights, advanced t..."
127,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Data Scientist,ClientSolv Technologies,Englewood,CO,2020-06-23 13:00:14.796878,NaN,Experience with some machine learning methodol...
128,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Statistician II,Aerotek,North Chicago,IL,2020-06-23 13:00:14.798874,NaN,"Study Preparation: Meets with CRA, R&D, Instru..."
129,https://www.indeed.com/rc/clk?jk=bddcb1faeb174...,Data Scientist - Production Research,"RiceTec, Inc.",Alvin,TX,2020-06-23 13:00:14.800868,NaN,The RiceTec Data Scientist will need to Design...
130,https://www.indeed.com/rc/clk?jk=85cc3860e666f...,Remote Sensing Data Scientist,University of Texas at Austin,Austin,TX,2020-06-23 13:00:14.803860,NaN,Demonstrated experience and familiarity in wor...
131,https://www.indeed.com/rc/clk?jk=d9b4bd00ee543...,Data Scientist - Shop Recommendations,Stitch Fix,San Francisco,CA,2020-06-23 13:00:14.806852,NaN,Address machine learning problems around recom...
132,https://www.indeed.com/company/Farm-Journal/jo...,Data Scientist,Farm Journal,Chicago,IL,2020-06-23 13:00:14.809844,NaN,"Provide data modeling, mining, pattern analysi..."
133,https://www.indeed.com/company/Methods-Consult...,Assistant Data Scientist,Methods Consultants,Ypsilanti,MI,2020-06-23 13:00:14.811838,NaN,"Competitive salary, health insurance, and reti..."
134,https://www.indeed.com/company/Altversity/jobs...,Data Science & Machine Learning Research Intern,Altversity,New York,NY,2020-06-23 13:00:14.814830,NaN,Has experience working in data science and mac...


In [50]:
df.to_csv(r'../Resources/initial.csv', index=False)